# A 10-minute Buy-and-Hold Template

This simple template provides basic instructions for helping you within 10 minutes to develop and submit a buy-and-hold strategy on liquid assets.

In [1]:
# Import basic libraries for manipulating data.

# Please refer to xarray.pydata.org for xarray documentation.

# xarray works optimally with N-dimensional datasets in Python
# and is well suited for financial datasets with labels "time",
# "field" and "asset". xarray data structures can also be easily
# converted to pandas dataframes.

import xarray as xr

import numpy as np
import pandas as pd

# Import quantiacs libraries.

import qnt.data as qndata          # data loading and manipulation
import qnt.stepper as qnstepper    # strategy definition
import qnt.stats as qnstats        # key statistics
import qnt.graph as qngraph        # graphical tools
#import qnt.forward_looking as qnfl # forward looking checking

# display function for fancy displaying:
from IPython.display import display

import datetime as dt

In [2]:
# Load all available asset names since given date.

assets = qndata.load_assets(tail = dt.timedelta(days=4*365))

assets_names = [i["id"] for i in assets]

# Load all available data since given date.

# It is possible to set a max_date in the call in order to
# develop the system on a limited in-sample period and later
# test the system on unseen data after max_date.

# A submission will be accepted only if no max_date is set,
# as submissions will be evaluated on live data on a daily basis.

data = qndata.load_data(tail = dt.timedelta(days=4*365),
                        dims=("time", "field", "asset"),
                        assets=assets_names,
                        forward_order=True)

fetched chunk 1/5 1s
fetched chunk 2/5 3s
fetched chunk 3/5 4s
fetched chunk 4/5 6s
fetched chunk 5/5 7s
Data loaded 7s


In [3]:
# A buy-and-hold strategy on liquid assets allocates
# constant fractions of capital to all liquid assets.
# Here xarray data structures are converted to pandas
# dataframes for simplicity in order to describe the
# development process.

# xarray.DataArray are converted to pandas dataframes:
is_liquid = data.loc[:,"is_liquid",:].to_pandas()

# set and normalize weights:
weights = is_liquid.div(is_liquid.abs().sum(axis=1, skipna=True), axis=0)
weights = weights.fillna(0.0)

# set max columns and rows for display function
with pd.option_context("display.max_rows", 5, "display.max_columns", 10): 
    display(weights)

asset,AMEX:APT,AMEX:IBIO,AMEX:IGC,AMEX:LNG,NASDAQ:AAL,...,NYSE:YUMC,NYSE:ZAYO,NYSE:ZBH,NYSE:ZEN,NYSE:ZTS
time,,,,,,,,,,,
2016-08-22,0.0,0.000000,0.0,0.002252,0.002252,...,0.0,0.002252,0.002252,0.000000,0.002252
2016-08-23,0.0,0.000000,0.0,0.002252,0.002252,...,0.0,0.002252,0.002252,0.000000,0.002252
...,...,...,...,...,...,...,...,...,...,...,...
2020-08-18,0.0,0.002105,0.0,0.000000,0.002105,...,0.0,0.000000,0.002105,0.002105,0.002105
2020-08-19,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000


In [4]:
# check that we are fully invested <=> sum of abs(weights) = 1:
with pd.option_context("display.max_rows", 5): 
    display(weights.abs().sum(axis=1))

time
2016-08-22    1.0
2016-08-23    1.0
             ... 
2020-08-18    1.0
2020-08-19    1.0
Length: 1006, dtype: float64

In [5]:
# visualize positions for a selected asset:
aapl_frac = weights["NASDAQ:AAPL"]
qngraph.make_plot(aapl_frac.index, aapl_frac, name= "aapl")

## Statistics

In [6]:
#convert to xarray before statistics calculation

output = weights.unstack().to_xarray()
output 

<xarray.DataArray (asset: 953, time: 1006)>
array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00210526, 0.00210526,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.00225225, 0.00225225, 0.00225225, ..., 0.00210526, 0.00210526,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00210526, 0.00210526,
        0.        ],
       [0.00225225, 0.00225225, 0.00225225, ..., 0.00210526, 0.00210526,
        0.        ]])
Coordinates:
  * asset    (asset) object 'AMEX:APT' 'AMEX:IBIO' ... 'NYSE:ZEN' 'NYSE:ZTS'
  * time     (time) datetime64[ns] 2016-08-22 2016-08-23 ... 2020-08-19

In [7]:
# Calculate statistics on a rolling basis.

# Transactions are punished with slippage equal to a given
# fraction of ATR14 (read more about slippage in our full
# Strategy Buy and Hold template). We evaluate submissions
# using 5% of ATR14 for slippage.

# Mean return, volatility and Sharpe ratio are computed on a
# rolling basis using a lookback period of 3 years.

stat = qnstats.calc_stat(data, output, slippage_factor=0.05)

display(stat.to_pandas().tail())

field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2020-08-13,1.559929,0.002090,0.238792,-0.000245,-0.382386,0.474300,0.113259,1.0,951.0,0.026524,94.990694
2020-08-14,1.558662,-0.000812,0.238743,-0.001057,-0.382386,0.459217,0.109635,1.0,951.0,0.026529,94.990694
2020-08-17,1.567330,0.005561,0.238754,0.000000,-0.382386,0.471985,0.112688,1.0,951.0,0.026528,94.990694
2020-08-18,1.563826,-0.002236,0.238757,-0.002236,-0.382386,0.464743,0.110961,1.0,951.0,0.026539,94.990694
2020-08-19,1.563794,-0.000020,0.238552,-0.002256,-0.382386,0.491177,0.117171,1.0,951.0,0.026520,194.440018


In [8]:
def print_stat(stat):
    """Prints selected statistical key indicators:
       - the global Sharpe ratio of the strategy;
       - the global mean profit;
       - the global volatility;
       - the maximum drawdown.

       Note that Sharpe ratio, mean profit and volatility
       apply to  max simulation period, and not to the
       rolling basis of 3 years.
    """

    days = len(stat.coords["time"])
    
    returns = stat.loc[:, "relative_return"]

    equity = stat.loc[:, "equity"]
    
    sharpe_ratio = qnstats.calc_sharpe_ratio_annualized(
        returns,
        max_periods=days,
        min_periods=days).to_pandas().values[-1]

    profit = (qnstats.calc_mean_return_annualized(
        returns,
        max_periods=days,
        min_periods=days).to_pandas().values[-1])*100.0

    volatility = (qnstats.calc_volatility_annualized(
        returns,
        max_periods=days,
        min_periods=days).to_pandas().values[-1])*100.0

    max_ddown = (qnstats.calc_max_drawdown(
        qnstats.calc_underwater(equity)).to_pandas().values[-1])*100.0

    print("Sharpe Ratio         : ", "{0:.3f}".format(sharpe_ratio))
    print("Mean Return [%]      : ", "{0:.3f}".format(profit))
    print("Volatility [%]       : ", "{0:.3f}".format(volatility))
    print("Maximum Drawdown [%] : ", "{0:.3f}".format(-max_ddown))

print_stat(stat)

Sharpe Ratio         :  0.562
Mean Return [%]      :  12.028
Volatility [%]       :  21.410
Maximum Drawdown [%] :  38.239


In [9]:
# show plot with profit and losses:
performance = stat.to_pandas()["equity"]
qngraph.make_plot_filled(performance.index, performance, name="PnL (Equity)", type="log")

In [10]:
# show underwater chart:
UWchart = stat.to_pandas()["underwater"]
qngraph.make_plot_filled(UWchart.index, UWchart, color="darkred", name="Underwater Chart", range_max=0)

In [11]:
# show rolling Sharpe ratio on a 3-year basis:
SRchart = stat.to_pandas()["sharpe_ratio"].iloc[(252*3):]
qngraph.make_plot_filled(SRchart.index, SRchart, color="#F442C5", name="Rolling SR")

In [12]:
# show bias chart:
biaschart = stat.to_pandas()["bias"]
qngraph.make_plot_filled(biaschart.index, biaschart, color="#5A6351", name="Bias Chart")

## Improvement

In [13]:
# Well, sharpe ratio of this strategy is low...

stat.sel(field='sharpe_ratio').to_pandas().tail()

time
2020-08-13    0.474300
2020-08-14    0.459217
2020-08-17    0.471985
2020-08-18    0.464743
2020-08-19    0.491177
dtype: float64

In [14]:
# Let's try to improve it using statistics per asset.
# At first, calculate stats per asset:

stat_per_asset = qnstats.calc_stat(data, output, slippage_factor=0.05, per_asset=True)
stat_per_asset.dims

('time', 'field', 'asset')

In [15]:
# For example, AAPL have good sharpe ratio:
stat_per_asset.sel(asset='NASDAQ:AAPL', field='sharpe_ratio').to_pandas().tail()

time
2020-08-13    1.271128
2020-08-14    1.254562
2020-08-17    1.240544
2020-08-18    1.253307
2020-08-19    1.274711
dtype: float64

In [16]:
# Let's build the output only with "good" "short term" and "long term" Sharpe ratios.
# Sharpe ratio is "good" when its average more then 0.
#
# This is only example of a heuristic which can improve you strategy 
# using statistics per asset. 
# I believe that you can invent a new better way to do it =)

short_term = 43
long_term = short_term*3

stat_per_asset_short_term = qnstats.calc_stat(data, output, max_periods=short_term, per_asset = True)
stat_per_asset_long_term = qnstats.calc_stat(data, output, max_periods=long_term, per_asset = True)

avg_short_term_sr = stat_per_asset_short_term.sel(field='sharpe_ratio')\
    .rolling(time=short_term, min_periods=short_term*19//20)\
    .mean() # min periods allows to pass small holes in data
avg_long_term_sr = stat_per_asset_long_term.sel(field='sharpe_ratio')\
    .rolling(time=long_term, min_periods=long_term*19//20)\
    .mean()

output2 = output
output2 = output2.where(avg_short_term_sr > 0)
output2 = output2.where(avg_long_term_sr > 0)
output2 = output2 / abs(output2).sum('asset')

In [17]:
# print stats

stat2 = qnstats.calc_stat(data, output2)

print("Old stats:")
print("-\n3y SR:")
print(stat.sel(field='sharpe_ratio').to_pandas().tail())
print("-\nGlobal:")
print_stat(stat)

print("---")

print("New stats:")
print("-\n3y SR:")
print(stat2.sel(field='sharpe_ratio').to_pandas().tail())
print("-\nGlobal:")
print_stat(stat2)

Old stats:
-
3y SR:
time
2020-08-13    0.474300
2020-08-14    0.459217
2020-08-17    0.471985
2020-08-18    0.464743
2020-08-19    0.491177
dtype: float64
-
Global:
Sharpe Ratio         :  0.562
Mean Return [%]      :  12.028
Volatility [%]       :  21.410
Maximum Drawdown [%] :  38.239
---
New stats:
-
3y SR:
time
2020-08-13    0.687691
2020-08-14    0.661286
2020-08-17    0.683726
2020-08-18    0.686304
2020-08-19    0.717552
dtype: float64
-
Global:
Sharpe Ratio         :  0.592
Mean Return [%]      :  12.936
Volatility [%]       :  21.851
Maximum Drawdown [%] :  33.653


Well, it only adds +0.18(\~23%) to the Sharpe ratio of 3 years, but adds +0.2(\~34%) to the Sharpe Ratio of the entire data period. Sometimes it can allow you to go through the Sharpe ratio filter or win a competition.

## Checks

In [18]:
# correlation check
# your strategy should not correlate with other strategies before submission
qnstats.print_correlation(output2, data)


Ok. This strategy does not correlate with other strategies.


## Write output

In [19]:
# Finally, we write the last mandatory step for submission,
# namely writing output to file:

qndata.write_output(output2)

write output: /root/fractions.nc.gz


At this stage code is ready for submission. Just click on the submission button in your account page and we will evaluate your strategy live on our servers!

For speeding up evaluation, you can consider submitting a copy with only relevant steps and excluding plots and checks. Your code in the final notebook you submit can be as simple as the following:

In [20]:
import xarray as xr
import qnt.data as qndata

# data loading
data = qndata.load_data(
    tail = dt.timedelta(days = 4*365),
    dims=("time", "field", "asset"), 
    forward_order=True
)

# buy and hold strategy output calculation
output1 = data.loc[::,"is_liquid",:]  
output1 = output1 / abs(output1).sum('asset')

# output improvement which use statistics per asset 
short_term = 43
long_term = short_term*3

stat_per_asset_short_term = qnstats.calc_stat(data, output1, max_periods=short_term, per_asset = True)
stat_per_asset_long_term = qnstats.calc_stat(data, output1, max_periods=long_term, per_asset = True)

avg_short_term_sr = stat_per_asset_short_term.sel(field='sharpe_ratio')\
    .rolling(time=short_term, min_periods=short_term*19//20)\
    .mean() # min periods allows to pass small holes in data
avg_long_term_sr = stat_per_asset_long_term.sel(field='sharpe_ratio')\
    .rolling(time=long_term, min_periods=long_term*19//20)\
    .mean()

output2 = output1
output2 = output2.where(avg_short_term_sr > 0)
output2 = output2.where(avg_long_term_sr > 0)
output2 = output2 / abs(output2).sum('asset')

qndata.write_output(output2)

fetched chunk 1/5 1s
fetched chunk 2/5 3s
fetched chunk 3/5 4s
fetched chunk 4/5 6s
fetched chunk 5/5 7s
Data loaded 7s
write output: /root/fractions.nc.gz
